In [1]:
import pandas as pd
from labels import LABELS
from dataset import EventDataset

import torch
from transformers import CamembertTokenizer, AutoTokenizer
from transformers import CamembertForSequenceClassification, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

In [2]:
data = "/teamspace/studios/this_studio/max/dataset_resume.csv"
#data = "./dataset_resume.csv"
#data = "./dataset_christian.csv"

try:
    df = pd.read_csv(data, usecols=LABELS + ['description', 'resume'])
except Exception as e:
    df = EventDataset(data_path=data).df

In [3]:
import numpy as np

df[LABELS] = df[LABELS].astype('float32')
#print(df.info())
def setup(df_orig: pd.DataFrame) -> pd.DataFrame:
    df = df_orig.copy()
    df = df.fillna(0)

    for index, row in df.iterrows():
        # s'il y a un résumé, on l'utilise comme description
        if 'resume' in row:
            df.loc[index, "description"] = (
                    row.resume
                    if isinstance(row.resume, str)
                    else row.description
                )

    df['label'] = df[LABELS].values.tolist()
    df = df.rename(columns={'description': 'text'})
    label = [list_label for list_label in df.label]
    return df.text.tolist(), label#.astype('float32')

features = df[LABELS]

texts, labels = setup(df)

In [5]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=.2, random_state=42)

In [6]:
checkpoint = "almanach/camembert-base"
tokenizer = CamembertTokenizer.from_pretrained(checkpoint)
model = CamembertForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels[0]), problem_type="multi_label_classification")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at almanach/camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
class EventDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = str(self.texts[index])
        label = torch.tensor(self.labels[index])

        encoding = self.tokenizer(text, 
                                  padding="max_length", 
                                  max_length=self.max_len,
                                  truncation=True,
                                  return_tensors="pt")

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': label
        }

In [8]:
train_ds = EventDataset(train_texts, train_labels, tokenizer)
val_ds = EventDataset(val_texts, val_labels, tokenizer)

In [9]:
# Multi-Label Classification Evaluation Metrics
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from transformers import EvalPrediction
import torch


def multi_labels_metrics(predictions, labels, threshold=0.3):
  sigmoid = torch.nn.Sigmoid()
  probs = sigmoid(torch.Tensor(predictions))

  y_pred = np.zeros(probs.shape)
  y_pred[np.where(probs>=threshold)] = 1
  y_true = labels

  f1 = f1_score(y_true, y_pred, average = 'macro')
  roc_auc = roc_auc_score(y_true, y_pred, average = 'macro')
  hamming = hamming_loss(y_true, y_pred)

  metrics = {
      "roc_auc": roc_auc,
      "hamming_loss": hamming,
      "f1": f1
  }

  return metrics

def compute_metrics(p:EvalPrediction):
  preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

  result = multi_labels_metrics(predictions=preds,
                                labels=p.label_ids)

  return result

In [10]:
# Training Arguments
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

args = TrainingArguments(
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    output_dir = './results',
    num_train_epochs=300,
    save_steps=1000,
    save_total_limit=2,
    eval_steps=10,
    evaluation_strategy = "epoch",
    save_strategy = 'epoch',
    load_best_model_at_end=True
)

trainer = Trainer(model=model,
                  args=args,
                  train_dataset=train_ds,
                  eval_dataset = val_ds, 
                  compute_metrics=compute_metrics,
                  callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
                  )

In [11]:
trainer.train()

Epoch,Training Loss,Validation Loss,Roc Auc,Hamming Loss,F1,Runtime,Samples Per Second,Steps Per Second
1,No log,0.323898,0.500000,0.088875,0.000000,0.919300,220.808000,18.491000
2,No log,0.284636,0.500000,0.088875,0.000000,0.920400,220.567000,18.471000
3,No log,0.276359,0.500000,0.088875,0.000000,0.921400,220.317000,18.450000
4,No log,0.258179,0.539691,0.082102,0.102023,0.917500,221.254000,18.529000
5,No log,0.237822,0.557825,0.080460,0.134351,0.921400,220.313000,18.450000
6,No log,0.219700,0.603333,0.074918,0.218055,0.920800,220.449000,18.461000
7,No log,0.206322,0.612850,0.072044,0.240616,0.924300,219.620000,18.392000
8,0.285800,0.191474,0.648290,0.069992,0.322521,0.930800,218.103000,18.265000
9,0.285800,0.179119,0.681836,0.063218,0.414450,0.919500,220.773000,18.488000
10,0.285800,0.170899,0.717970,0.060961,0.483909,0.927100,218.953000,18.336000


TrainOutput(global_step=2448, training_loss=0.108210857592377, metrics={'train_runtime': 483.3206, 'train_samples_per_second': 502.772, 'train_steps_per_second': 42.208, 'total_flos': 3836916940308480.0, 'train_loss': 0.108210857592377, 'epoch': 36.0})

In [12]:
trainer.evaluate()

{'eval_loss': 0.10126499831676483,
 'eval_roc_auc': 0.9012017343198472,
 'eval_hamming_loss': 0.03284072249589491,
 'eval_f1': 0.8243244695138809,
 'eval_runtime': 0.9185,
 'eval_samples_per_second': 221.001,
 'eval_steps_per_second': 18.507,
 'epoch': 36.0}

In [13]:
trainer.save_model("/teamspace/studios/this_studio/max/camembert-tourism-events")

In [ ]:
# import pickle
# with open("/teamspace/studios/this_studio/max/features.pkl", "wb") as f:
#   pickle.dump(features, f)

In [14]:
text = """Après de multiples tournées à succès et plus de 2 millions de billets vendus, Messmer, connu et reconnu comme le Maître Mondial de l’Hypnose revient près de chez vous !

Dans 13Hz, Messmer vous invite à entrer dans son mystérieux et hilarant univers où la frontière entre la réalité et l’illusion s’efface, pour diriger vos pensées vers des territoires inconnus.

Avec sa présence charismatique inégalée et son talent exceptionnel, le recordman en hypnose collective avec 1066 personnes hypnotisées en moins de 5 minutes, vous plonge au cœur de vos pensées les plus profondes avant de vous guider à travers un jeu subtil d’ondes cérébrales à 13Hz.

Le fascinateur vous entraîne vers un état de conscience unique où la volonté et le contrôle de nos vies prennent une nouvelle dimension.

Osez découvrir l’expérience Messmer, où la maîtrise de soi et la fascination se rencontrent."""

encoding = tokenizer(text, return_tensors='pt')
encoding.to(trainer.model.device)

outputs = trainer.model(**encoding)

In [15]:
import json
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0].cpu()).detach().numpy()
preds = np.zeros(probs.shape)
##preds[np.where(probs>=0.3)] = 1


def sort_dict_by_value(dict1):
  """Trie un dictionnaire par ordre décroissant des valeurs."""
  sorted_dict = {x: y for x, y in sorted(dict1.items(), key=lambda item: item[1], reverse=True)}
  return sorted_dict
  
preds = '{'
for idx, label in enumerate(LABELS):
    preds += f'"{label}": {probs[idx]},'

preds = preds.strip(',')
preds += '}'
preds = json.loads(preds)
sort_dict_by_value(preds)

{'Spectacle': 0.978179395198822,
 'Détente': 0.5542797446250916,
 'Famille': 0.22902777791023254,
 'Théatre': 0.08246546983718872,
 'Danse': 0.06862802058458328,
 'Culture': 0.0195869579911232,
 'Jeu': 0.018659785389900208,
 'Art': 0.012486935593187809,
 'Atelier': 0.01069662906229496,
 'Santé': 0.009508967399597168,
 'Concert': 0.007548123598098755,
 'Festival': 0.004736847709864378,
 'Exposition': 0.002396058291196823,
 'Sport': 0.0017266126815229654,
 'Visite': 0.0014010719023644924,
 'Brocante': 0.000987489358521998,
 'Marché': 0.0008547356119379401,
 'Action': 0.0008104487787932158,
 'Fête': 0.0005079141701571643,
 'Histoire': 0.00040898207225836813,
 'Gastronomie': 0.00035391654819250107,
 'Conférence': 0.00028567150002345443,
 'Balade': 0.00012177215830888599,
 'Environnement': 9.488358773523942e-05}

# Utilisation d'un modèle enregistré en local

In [23]:
import json
import torch 
import numpy as np
from labels import LABELS
from transformers import CamembertForSequenceClassification, CamembertTokenizer, Trainer

checkpoint = "almanach/camembert-base"
path = '/teamspace/studios/this_studio/max/camembert-tourism-events'
path = './camembert-tourism-events'
tokenizer = CamembertTokenizer.from_pretrained(checkpoint)
model = CamembertForSequenceClassification.from_pretrained(path)
trainer = Trainer(model=model)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
text = """Après de multiples tournées à succès et plus de 2 millions de billets vendus, Messmer, connu et reconnu comme le Maître Mondial de l’Hypnose revient près de chez vous !

Dans 13Hz, Messmer vous invite à entrer dans son mystérieux et hilarant univers où la frontière entre la réalité et l’illusion s’efface, pour diriger vos pensées vers des territoires inconnus.

Avec sa présence charismatique inégalée et son talent exceptionnel, le recordman en hypnose collective avec 1066 personnes hypnotisées en moins de 5 minutes, vous plonge au cœur de vos pensées les plus profondes avant de vous guider à travers un jeu subtil d’ondes cérébrales à 13Hz.

Le fascinateur vous entraîne vers un état de conscience unique où la volonté et le contrôle de nos vies prennent une nouvelle dimension.

Osez découvrir l’expérience Messmer, où la maîtrise de soi et la fascination se rencontrent."""
encoding = tokenizer(text, return_tensors='pt')
encoding.to(trainer.model.device)

outputs = trainer.model(**encoding)

In [28]:
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0].cpu()).detach().numpy()
preds = np.zeros(probs.shape)
##preds[np.where(probs>=0.3)] = 1


def sort_dict_by_value(dict1):
  """Trie un dictionnaire par ordre décroissant des valeurs."""
  sorted_dict = {x: y for x, y in sorted(dict1.items(), key=lambda item: item[1], reverse=True)}
  return sorted_dict
  
preds = '{'
for idx, label in enumerate(LABELS):
    preds += f'"{label}": {probs[idx]},'

preds = preds.strip(',')
preds += '}'
preds = json.loads(preds)
sort_dict_by_value(preds)

{'Spectacle': 0.978179395198822,
 'Détente': 0.5542798042297363,
 'Famille': 0.22902809083461761,
 'Théatre': 0.08246543258428574,
 'Danse': 0.06862791627645493,
 'Culture': 0.019586967304348946,
 'Jeu': 0.01865977980196476,
 'Art': 0.012486924417316914,
 'Atelier': 0.01069661881774664,
 'Santé': 0.00950896181166172,
 'Concert': 0.007548119872808456,
 'Festival': 0.004736841190606356,
 'Exposition': 0.0023960559628903866,
 'Sport': 0.001726614311337471,
 'Visite': 0.0014010719023644924,
 'Brocante': 0.000987487961538136,
 'Marché': 0.0008547352044843137,
 'Action': 0.0008104483713395894,
 'Fête': 0.0005079141701571643,
 'Histoire': 0.0004089822468813509,
 'Gastronomie': 0.00035391654819250107,
 'Conférence': 0.00028567150002345443,
 'Balade': 0.00012177215830888599,
 'Environnement': 9.488358773523942e-05}